# APE 26 Prototype: Removing data storage (representations) from coordinate frames

This prototype represents the _final_ stage of implementation proposed in [APE 26](https://github.com/jeffjennings/astropy-APEs/blob/APE26/APE26.rst) ([PR](https://github.com/astropy/astropy-APEs/pull/112)). Thus the following changes are treated as having already occurred:

- Frame classes in `coordinates` were split into two hierarchies: ones with and without data, with the data-less ones getting new names (e.g., `ICRSFrame` instead of `ICRS`; demonstrated below).
- A new `Coordinate` class was added (demonstrated below).
- `SkyCoord` was switched to use only the data-less frame classes (demonstrated below).
- After a deprecation cycle, the legacy, with-data frame classes were removed (and are thus only included below for comparison).

Below we demonstrate:
1) [the structure and use of the new frame classes](#frames),
2) [the structure and use of the new `Coordinate` class](#coordinate),
3) [changes to `SkyCoord`](#skycoord), 
4) [the APE's improved separation of concerns and clearer API](#soc), and
5) [practical benefits of these implementations relative to current API](#benefits).

In [2]:
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import representation as r
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord as SkyCoord_legacy

from ape26_prototype import (
    BaseFrame, ICRSFrame, FK5Frame, GalacticFrame,
    BaseCoordinate, Coordinate, SkyCoord,
)

<a id='frames'></a>
## 1) The new frame classes

New coordinate frames (e.g. `ICRSFrame`, `FK5Frame`): 
- subclass the new `BaseFrame` (instead of the previous `BaseCoordinateFrame`)
- no longer hold coordinate data (`Representation` or `Differential` objects, or any 
associated methods), and thus: 
    - no longer have duplicated functionality, duplicated code or duplicated testing with `SkyCoord`
    - no longer have different usage modes (with vs. without data) that exhibit different behavior and require code interacting with the frame classes to handle both cases (checking `frame.has_data`)
    - static analyzers can now perform more useful type checking
    - `BaseFrame` correspondingly does not hold coordinate data (unlike the legacy `BaseCoordinateFrame`)

In [3]:
# initialize new frames
icrs = ICRSFrame()
fk5_j2000 = FK5Frame(equinox=Time("J2000"))
fk5_j1950 = FK5Frame(equinox=Time("J1950"))
gal = GalacticFrame()

print("New frame objects (no data):")
print(f"  ICRS frame: {icrs}")
print(f"  Galactic frame: {gal}")
print(f"  FK5 J2000 frame: {fk5_j2000}")
print(f"  FK5 J1950 frame: {fk5_j1950}")
print(f"    FK5 J1950 frame attributes: {fk5_j1950._get_frame_attributes()}")

# compare frames
print(f"\nFrame equivalence:")
print(f"  FK5 J2000 == FK5 J2000: {fk5_j2000.is_equivalent_frame(fk5_j2000)}")
print(f"  FK5 J2000 == FK5 J1950: {fk5_j2000.is_equivalent_frame(fk5_j1950)}")

# show subclassing
print(f"\nFrame inheritance:")
print(f"  ICRSFrame is a subclass of: {ICRSFrame.__bases__[0]}")
print(f"  FK5Frame is a subclass of BaseFrame: {issubclass(FK5Frame, BaseFrame)}")
print(f"  GalacticFrame is a subclass of BaseFrame: {issubclass(GalacticFrame, BaseFrame)}")

New frame objects (no data):
  ICRS frame: ICRSFrame()
  Galactic frame: GalacticFrame()
  FK5 J2000 frame: <FK5Frame (equinox=J2000.000)>
  FK5 J1950 frame: <FK5Frame (equinox=J1950.000)>
    FK5 J1950 frame attributes: {'equinox': <Time object: scale='tt' format='jyear_str' value=J1950.000>}

Frame equivalence:
  FK5 J2000 == FK5 J2000: True
  FK5 J2000 == FK5 J1950: False

Frame inheritance:
  ICRSFrame is a subclass of: <class 'ape26_prototype.BaseFrame'>
  FK5Frame is a subclass of BaseFrame: True
  GalacticFrame is a subclass of BaseFrame: True


<a id='coordinate'></a>
## 2) The new `Coordinate` class

`Coordinate` is a new, bare bones alternative to `SkyCoord`. It:
 
- like `SkyCoord`, represents data (using `Representation` and `Differential` classes) in a 
  given reference frame
    - both `Coordinate` and `SkyCoord` subclass the new class `BaseCoordinate`
- like the APE's updated version of `SkyCoord` (see below), only accepts data-less frame classes
- unlike `SkyCoord`, does not keep any frame attributes not in the current frame, and does not have extra features 
  like caching and flexible input parsing
    - it is thus more lightweight (few internal methods) and performant 
- is a direct replacement for `BaseCoordinateFrame` (and operates very similarly to `BaseCoordinateFrame` objects 
        when they held data)

In [4]:
# Create some coordinate data
data = r.SphericalRepresentation(
    lon=10.0 * u.deg,
    lat=20.0 * u.deg,
    distance=1.0 * u.kpc
)

# Create a new Coordinate object (frame + data)
icrs = ICRSFrame()
c = Coordinate(frame=icrs, data=data)
print(f"Coordinate object:\n {c}\n\tFrame: {c.frame}\n\tData: {c.data}")

c_new = c.__replace__(data=data * 5)
print(f"\nUsing replacement, we alter the distance:\n {c_new}")
print(f"The original class instance is unchanged:\n {c}")

# Represent in Cartesian coordinates, using Coordinate
data_cartesian = c.data.represent_as(r.CartesianRepresentation)
print(f"\nChange representation to Cartesian:\n {data_cartesian}")

# Represent in Cartesian coordinates, using SkyCoord
sc = SkyCoord(lon=10.0 * u.deg, lat=20.0 * u.deg, distance=1.0 * u.kpc, frame='icrs')
cartesian = sc.cartesian  
print(f"\nCreate the analogous object with `SkyCoord`:\n {sc}")
print(f"...and change its representation to Cartesian:\n {cartesian}")

# Transform the Coordinate oebject from ICRSFrame to FK5Frame
fk5 = FK5Frame(equinox=Time("J1950"))
c_fk5 = c.transform_to(fk5)     
print(f"\nTransform the `Coordinate` instance from ICRS to FK5:\n {c_fk5}")

# Transform between `FK5Frame`s with different equinoxes
fk5_j2000 = FK5Frame(equinox=Time("J2000"))
c_fk5_j2000 = c_fk5.transform_to(fk5_j2000)
print(f"...and transform (precess) the FK5 instance to another equinox:\n {c_fk5_j2000}")

Coordinate object:
 <Coordinate (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>
	Frame: ICRSFrame()
	Data: (10., 20., 1.) (deg, deg, kpc)

Using replacement, we alter the distance:
 <Coordinate (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 5.)>>
The original class instance is unchanged:
 <Coordinate (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>

Change representation to Cartesian:
 (0.92541658, 0.16317591, 0.34202014) kpc

Create the analogous object with `SkyCoord`:
 <SkyCoord (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>
...and change its representation to Cartesian:
 (0.92541658, 0.16317591, 0.34202014) kpc

Transform the `Coordinate` instance from ICRS to FK5:
 <Coordinate (FK5Frame): <CartesianRepresentation (x, y, z) in kpc
    (0.92541655, 0.16317605, 0.34202017)>>
...and transf

In [5]:
# Similar to the above, but showing cylindrical coordinates also work
data_cyl = r.CylindricalRepresentation(
    rho=180.0 * u.pc,
    phi=60.0 * u.deg,
    z=100.0 * u.pc
)

# Create coordinate in Galactic frame
c_cyl = Coordinate(frame=GalacticFrame(), data=data_cyl)
print(f"\nCoordinate object:\n {c_cyl}")

# Transform to ICRS frame
cyl_icrs = c_cyl.transform_to(ICRSFrame())
print(f"\nTransform to ICRS:\n {cyl_icrs}")

data_car = c_cyl.data.represent_as(r.CartesianRepresentation)
print(f"\nChange representation to Cartesian:\n {data_car}")


Coordinate object:
 <Coordinate (GalacticFrame): <CylindricalRepresentation (rho, phi, z) in (pc, deg, pc)
    (180., 60., 100.)>>

Transform to ICRS:
 <Coordinate (ICRSFrame): <CartesianRepresentation (x, y, z) in pc
    (-14.68138448, -167.75905941, 118.49622329)>>

Change representation to Cartesian:
 (90., 155.88457268, 100.) pc


<a id='skycoord'></a>
# 3) Updates to `SkyCoord`

Changes to `SkyCoord` in this APE result in a near-identical user API, while internally it: 
  - now uses only data-less frame classes, reducing code duplication between it and frame classes
  - now has its methods and attributes defined directly on it, 
  no longer needing to dynamically call `BaseCoordinateFrame` via `SkyCoord.__getattr__` 
  to manage coordinate data
  - has had the bulk of its methods (those that should be accessible also to `Coordinate`) moved onto `BaseCoordinate`


In [6]:
# Create a SkyCoord object - unchanged API
sc1 = SkyCoord(10*u.deg, 20*u.deg, distance=1*u.kpc, frame='icrs')
print(f"Created SkyCoord (same as current API):")
print(f"  {sc1}")

# Show internal structure
print(f"\nInternal structure:")
print(f"  Frame type (new): {type(sc1.frame)}")
print(f"  Frame (new): {sc1.frame}")
print(f"  Data type (unchanged): {type(sc1.data)}")
print(f"  Data (unchanged): {sc1.data}")

# Transform to another frame - unchanged API
sc2 = sc1.transform_to('fk5')
print(f"\nTransform the SkyCoord object to FK5 (same as current API):")
print(f"  {sc2}")
print(f"  Equinox: {sc2.frame.equinox}")

# Calculate separation - unchanged API
sc3 = SkyCoord(15*u.deg, 25*u.deg, frame='icrs')
sep = sc1.separation(sc3)
print(f"\nCompute separation (same as current API): {sep}")

Created SkyCoord (same as current API):
  <SkyCoord (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>

Internal structure:
  Frame type (new): <class 'ape26_prototype.ICRSFrame'>
  Frame (new): ICRSFrame()
  Data type (unchanged): <class 'astropy.coordinates.representation.spherical.SphericalRepresentation'>
  Data (unchanged): (10., 20., 1.) (deg, deg, kpc)

Transform the SkyCoord object to FK5 (same as current API):
  <SkyCoord (FK5Frame): <CartesianRepresentation (x, y, z) in kpc
    (0.92541655, 0.16317605, 0.34202017)>>
  Equinox: J2000.000

Compute separation (same as current API): 6.80561007516086 deg


<a id='soc'></a>
# 4) Separation of concerns and a clearer API

One of the largest benefits of the APE's changes to users is a clearer API; there is one 
obvious way to store coordinate data, in `SkyCoord` or its bare bones analog `Coordinate`. 
There is no longer a second way - in frame classes - which had introduced a different object with a slightly 
different API.

In [7]:
print("CURRENT astropy.coordinates: two ways to store the same data:")
# ...using a frame class
c1 = coord.ICRS(ra=150*u.deg, dec=-11*u.deg)
print("  Method 1 - frame with data:")
print(f"    {c1}")
print(f"    Type: {type(c1).__name__}")
print(f"    Has .separation(): {hasattr(c1, 'separation')}")

# ...and using SkyCoord
c2 = coord.SkyCoord(ra=150*u.deg, dec=-11*u.deg, frame='icrs')
print("  Method 2 - SkyCoord:")
print(f"    {c2}")
print(f"    Type: {type(c2).__name__}")
print(f"    Has .separation(): {hasattr(c2, 'separation')}")

print("\nThis results in different types with similar but not identical APIs:")
print(f"  Equivalency:  ICRS object == SkyCoord object: {c1 == c2}")
print(f"  But no type equivalency:  type(ICRS object) == type(SkyCoord object): {type(c1) == type(c2)}")

CURRENT astropy.coordinates: two ways to store the same data:
  Method 1 - frame with data:
    <ICRS Coordinate: (ra, dec) in deg
    (150., -11.)>
    Type: ICRS
    Has .separation(): True
  Method 2 - SkyCoord:
    <SkyCoord (ICRS): (ra, dec) in deg
    (150., -11.)>
    Type: SkyCoord
    Has .separation(): True

This results in different types with similar but not identical APIs:
  Equivalency:  ICRS object == SkyCoord object: True
  But no type equivalency:  type(ICRS object) == type(SkyCoord object): False


In [8]:
print("\nContrast the above with the APE framework:")
print("\nWe have a data-less frame:")
frame_ape = ICRSFrame()
print(f"  {frame_ape}")
print(f"  Type: {type(frame_ape).__name__}")
print(f"  Has .separation(): {hasattr(frame_ape, 'separation')}")

print("\nWe create a `Coordinate` object, combining our frame with data:")
data_ape = r.SphericalRepresentation(
    lon=150*u.deg, lat=-11*u.deg, distance=1*u.kpc
)
c1_ape = Coordinate(frame=frame_ape, data=data_ape)
print(f"  {c1_ape}")
print(f"  Type: {type(c1_ape).__name__}")
print(f"  Has .separation(): {hasattr(c1_ape, 'separation')}")

print("\nOr we can use SkyCoord, as usual:")
c2_ape = SkyCoord(ra=150*u.deg, dec=-11*u.deg, frame='icrs')
print(f"  {c2_ape}")
print(f"  Type: {type(c2_ape).__name__}")
print(f"  Has .separation(): {hasattr(c2_ape, 'separation')}")

print(f"\n`Coordinate` and `SkyCoord` objects they are both instances of `BaseCoordinate`:")
print(f"  isinstance(Coordinate object, BaseCoordinate): {isinstance(c2_ape, BaseCoordinate)}")
print(f"  isinstance(SkyCoord object, BaseCoordinate): {isinstance(c2_ape, BaseCoordinate)}")

print("And thus they share many data-ful attributes:")
print(f"  Shape, size of Coordinate object: {c1_ape.shape, c1_ape.size}")
print(f"  Shape, size of SkyCoord object: {c2_ape.shape, c2_ape.size}")
print(f"  Separation between Coordinate and SkyCoord objects: {c1_ape.separation(c2_ape)}")



Contrast the above with the APE framework:

We have a data-less frame:
  ICRSFrame()
  Type: ICRSFrame
  Has .separation(): False

We create a `Coordinate` object, combining our frame with data:
  <Coordinate (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (150., -11., 1.)>>
  Type: Coordinate
  Has .separation(): True

Or we can use SkyCoord, as usual:
  <SkyCoord (ICRSFrame): <UnitSphericalRepresentation (lon, lat) in deg
    (150., -11.)>>
  Type: SkyCoord
  Has .separation(): True

`Coordinate` and `SkyCoord` objects they are both instances of `BaseCoordinate`:
  isinstance(Coordinate object, BaseCoordinate): True
  isinstance(SkyCoord object, BaseCoordinate): True
And thus they share many data-ful attributes:
  Shape, size of Coordinate object: ((), 1)
  Shape, size of SkyCoord object: ((), 1)
  Separation between Coordinate and SkyCoord objects: 0.0 deg


<a id='benefits'></a>
# 5) Practical benefits of the new framework

A few common use cases below demonstrate the advantages of the APE's final implementation.

In [9]:
print(f"Example 1: save memory by re-using a frame definition without copying the data:")
# create a single frame 
my_frame = FK5Frame(equinox=Time("J1950"))
# re-use it with different data
data1 = r.SphericalRepresentation(lon=10*u.deg, lat=20*u.deg, distance=1*u.kpc)
data2 = r.SphericalRepresentation(lon=15*u.deg, lat=25*u.deg, distance=2*u.kpc)
coord1 = Coordinate(frame=my_frame, data=data1)
coord2 = Coordinate(frame=my_frame, data=data2)

print(f"  Frame: {my_frame}")
print(f"  Coordinate 1:\n    {coord1}")
print(f"  Coordinate 2:\n    {coord2}")
print(f"  Same frame instance: {coord1.frame is coord2.frame}")

Example 1: save memory by re-using a frame definition without copying the data:
  Frame: <FK5Frame (equinox=J1950.000)>
  Coordinate 1:
    <Coordinate (FK5Frame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>
  Coordinate 2:
    <Coordinate (FK5Frame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (15., 25., 2.)>>
  Same frame instance: True


In [10]:
print(f"\nExample 2: improve type checking and docs with type-safe frame transformations:")
icrs_coord = Coordinate(
    frame=ICRSFrame(),
    data=r.SphericalRepresentation(lon=10*u.deg, lat=20*u.deg, distance=1*u.kpc)
)
fk5_coord = icrs_coord.transform_to(FK5Frame(equinox="J1900"))
print(f"  `Coordinate` object in ICRS:\n\t{icrs_coord}")
print(f"  Its type annotations: {coord1.__annotations__}")
print(f"  Inspect needed arguments to transform to an FK5 frame: {FK5Frame.__annotations__}")
print(f"  Transform to FK5:\n\t{fk5_coord}")


Example 2: improve type checking and docs with type-safe frame transformations:
  `Coordinate` object in ICRS:
	<Coordinate (ICRSFrame): <SphericalRepresentation (lon, lat, distance) in (deg, deg, kpc)
    (10., 20., 1.)>>
  Its type annotations: {'data': 'BaseRepresentation', 'frame': 'BaseFrame'}
  Inspect needed arguments to transform to an FK5 frame: {'equinox': 'Time'}
  Transform to FK5:
	<Coordinate (FK5Frame): <CartesianRepresentation (x, y, z) in kpc
    (0.92541655, 0.16317605, 0.34202017)>>


In [11]:
print(f"\nExample 3: perform frame operations without data (e.g., assess equivalency):")
f1 = FK5Frame(equinox=Time("J2000"))
f2 = FK5Frame(equinox=Time("J2000"))
f3 = FK5Frame(equinox=Time("J1950"))

print(f"  Frame 1: {f1}")
print(f"  Frame 2 (same equinox): {f2}")
print(f"  Frame 3 (different equinox): {f3}")
print(f"  F1 equivalency with F2: {f1.is_equivalent_frame(f2)}")
print(f"  F1 equivalency with F3: {f1.is_equivalent_frame(f3)}")


Example 3: perform frame operations without data (e.g., assess equivalency):
  Frame 1: <FK5Frame (equinox=J2000.000)>
  Frame 2 (same equinox): <FK5Frame (equinox=J2000.000)>
  Frame 3 (different equinox): <FK5Frame (equinox=J1950.000)>
  F1 equivalency with F2: True
  F1 equivalency with F3: False


In [12]:
print(f"\nExample 4: improve memory and performance when creating many data-ful objects:")
# frame is created once
frame = ICRSFrame()
n = 1000
# Create 1000 instances with differing data
coords = []
for i in range(n):
    data = r.SphericalRepresentation(
        lon=(i % 360) * u.deg,
        lat=((i % 180) - 90) * u.deg,
        distance=1 * u.kpc
    )
    coords.append(Coordinate(frame=frame, data=data))

print(f"  Initialized a single frame instance: {frame}")
print(f"    ...and used it to create {len(coords)} `Coordinate` objects with different data.")
print(f"    All `Coordinate` objects share the same frame instance: {all(c.frame is frame for c in coords)}")
print(f"  This single-frame operation was thus memory-efficient and fast.")
print(f"  It's also cache-friendly when using `SkyCoord`, because frame transformations only get cached once.")


Example 4: improve memory and performance when creating many data-ful objects:
  Initialized a single frame instance: ICRSFrame()
    ...and used it to create 1000 `Coordinate` objects with different data.
    All `Coordinate` objects share the same frame instance: True
  This single-frame operation was thus memory-efficient and fast.
  It's also cache-friendly when using `SkyCoord`, because frame transformations only get cached once.


In [13]:
print(f"\nExample 5: `Coordinate` is highly performant relative to `SkyCoord`:")
frame = ICRSFrame()
data = r.SphericalRepresentation(lon=10*u.deg, lat=20*u.deg, distance=1*u.kpc)

print("Timing `Coordinate` initialization:")
%timeit Coordinate(frame=frame, data=data)
print("Timing `SkyCoord` initialization:")
%timeit SkyCoord_legacy(data, frame="icrs")


Example 5: `Coordinate` is highly performant relative to `SkyCoord`:
Timing `Coordinate` initialization:
319 ns ± 3.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Timing `SkyCoord` initialization:
57.6 μs ± 292 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
